Qiita [Db2 v12.1.2でベクトルデータが入るようになったので、SQLでいろいろやってみた](https://qiita.com/nishikyon/items/8fdb36aaffd59e617418)の「1. (前準備)テーブル準備」のCSVファイル作成部分です

# 01. 万博情報データ作成

Xより　@takacho_01　さんが公開している<br>
【大阪万博のパビリオンまとめ表】<br>
（2025.07.29更新／完成版）<br>
- https://x.com/takacho_01/status/1950160001059672529
- https://x.com/takacho_01/status/1950181115446071371

こちらのデータを取得します.

尚、個人の方が公開しているファイルのため、今後(万博終了後?)取得できなく可能性が高いです

In [ ]:
expo_info_file="EXPO2025／海外パビリオン一覧.xlsx"

In [ ]:
expo_info_file="EXPO2025／海外パビリオン一覧.xlsx"
!wget "https://docs.google.com/spreadsheets/d/1lMaWkOekdDhxWcCQfAsVI5tgjq_J8hq_wqfqWsDpAPE/export?format=xlsx" -O {expo_info_file}


Db2に読み込めるように加工します！

今後フォーマット変更された場合は修正の必要があります

In [ ]:
import pandas as pd

# ヘッダー行のインデックス（3行目）
header_row = 2

# 各シートの読み込み制限行数
sheet_limits = {
    "海外パビリオン": 65 - header_row -1 ,
    "シグネチャー": 16 - header_row -1,
    "企業・イベント系": 27 - header_row -1
}


# 各シートを指定行数まで読み込み
sheet_dfs = {
    sheet_name: pd.read_excel(expo_info_file, sheet_name=sheet_name, header=header_row, nrows=sheet_limits[sheet_name], engine="openpyxl")
    for sheet_name in sheet_limits
}

# 「シグネチャー」シートの「詳細」列に「（5段階）」列の値を追記し、「（5段階）」列を削除
signature_df = sheet_dfs.get("シグネチャー")
if signature_df is not None and "詳細" in signature_df.columns and "（5段階）" in signature_df.columns:
    signature_df["詳細"] = signature_df["詳細"].astype(str) + "\n難易度5段階: " + signature_df["（5段階）"].astype(str)
    signature_df.drop(columns=["（5段階）"], inplace=True)
    signature_df.insert(4, "混雑度", '')
    

# 統一ヘッダー
new_headers = [
    "パビリオン名", "主観評価", "事前予約", "見学タイプ", "混雑度", "見どころ", "詳細",
    "おとな向け評価", "おとなコメント", "中高生向け評価", "中高生コメント",
    "子連れ向け評価", "子連れコメント"
]

for sheet_name, df in sheet_dfs.items():
    if len(df.columns) == len(new_headers):
        df.columns = new_headers
      

# カテゴリー列を追加
for sheet_name, df in sheet_dfs.items():
    df.insert(0, "カテゴリー", sheet_name)

# 全シート統合
combined_df = pd.concat(sheet_dfs.values(), ignore_index=True)





# 改行コードをスペースに置換（文字列型のみ処理）
for col in combined_df.columns:
    combined_df[col] = combined_df[col].apply(
        lambda x: x.replace('\n', ' ') if isinstance(x, str) else x
    )



# ID列を追加（001からの連番文字列）
combined_df.insert(0, "ID", [f"ID{i:03d}" for i in range(1, len(combined_df) + 1)])


# CSVとついでにExcelに保存
combined_df.to_csv("expo_info.csv", index=False, encoding="utf-8")
combined_df.to_excel("expo_info.xlsx", index=False)
